# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Modify-ecoinvent" data-toc-modified-id="Modify-ecoinvent-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modify ecoinvent</a></div><div class="lev1 toc-item"><a href="#Prepare-additional-datasets" data-toc-modified-id="Prepare-additional-datasets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare additional datasets</a></div><div class="lev2 toc-item"><a href="#import-from-excel" data-toc-modified-id="import-from-excel-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>import from excel</a></div><div class="lev1 toc-item"><a href="#Import-new-datasets-and-ecoinvent-into-Wurst-format" data-toc-modified-id="Import-new-datasets-and-ecoinvent-into-Wurst-format-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import new datasets and ecoinvent into Wurst format</a></div><div class="lev1 toc-item"><a href="#Define-REMIND-scenarios-and-years-to-be-used:" data-toc-modified-id="Define-REMIND-scenarios-and-years-to-be-used:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Define REMIND scenarios and years to be used:</a></div><div class="lev1 toc-item"><a href="#Modify-all-electricity-generation-and-market-datasets:" data-toc-modified-id="Modify-all-electricity-generation-and-market-datasets:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modify all electricity generation and market datasets:</a></div>

# Modify ecoinvent

In this notebook we actually run the functions that we've been creating so that we can modify ecoinvent!

We create 3 new versions of ecoinvent.

1. ecoinvent 2012, which has updates based on image results from 2012
2. ecoinvent 2040 BAU, which is a 2040 version of ecoinvent based on the image BAU scenario
3. ecoinvent 2040 450, which is a 2040 version of ecoinvent based on the image 450 ppm scenario


In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
%matplotlib inline

Import useful libraries and set project:

In [3]:
%run initialize_notebook.ipynb

Import functions that are written on top of Wurst:

In [5]:
%run REMIND_Functions_to_modify_ecoinvent.ipynb

Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

# Prepare additional datasets

We import some additional datasets for processes such as CSP and carbon capture and storage electricity. These have been prepared in Simapro and exported. We have cleaned up the datasets and stored them in excel using Brightway input output functionality. We read them from excel here:

## import from excel

In [6]:
for k, fp in {'Carma CCS': 'lci-Carma-CCS.xlsx'}.items():
   
    if k not in databases:
        
        sp = ExcelImporter(fp)
        sp.apply_strategies()
        sp.match_database(fields=["name", "unit", "location"]) 
        sp.match_database('ecoinvent_3.5',  fields=["reference product", "name", "unit", "location"]) 
        sp.match_database('ecoinvent_3.5',  fields=[ "name", "unit", "location"])
        sp.write_database()
        del sp
        
        lca = LCA({Database(k).random(): 1}, method=lcia_methods['CC'])
        lca.lci()
        lca.lcia()
        print(lca.score)

Extracted 1 worksheets in 0.38 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.28 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/08/2018 11:08:23
  Finished: 11/08/2018 11:08:23
  Total time elapsed: 00:00:00
  CPU %: 63.20
  Memory %: 1.83
Created database: Carma CCS
0.004134829852195358


# Import new datasets and ecoinvent into Wurst format

We are using the constructive geometries library (by Chris Mutel), which is still in it's infancy. There are some naming inconsistencies still, which we correct for in the following cell:

# Define REMIND scenarios and years to be used:

Here we set the scenarios and years for which we want to use when creating a new version of ecoinvent:

In [7]:
database_dict = collections.defaultdict(dict)

for year in [2030, 2050]:
    for scenario in ['BAU','RCP26','RCP37']:
        if year == 2015 and scenario != 'BAU':
            continue
        db_name = 'ecoinvent_Remind_' + scenario + '_' + str(year)
        database_dict[db_name] = {'year' : year , 'scenario' : scenario}


In [9]:
database_dict

defaultdict(dict,
            {'ecoinvent_Remind_BAU_2030': {'year': 2030, 'scenario': 'BAU'},
             'ecoinvent_Remind_RCP26_2030': {'year': 2030,
              'scenario': 'RCP26'},
             'ecoinvent_Remind_RCP37_2030': {'year': 2030,
              'scenario': 'RCP37'},
             'ecoinvent_Remind_BAU_2050': {'year': 2050, 'scenario': 'BAU'},
             'ecoinvent_Remind_RCP26_2050': {'year': 2050,
              'scenario': 'RCP26'},
             'ecoinvent_Remind_RCP37_2050': {'year': 2050,
              'scenario': 'RCP37'}})

In [8]:
databases

Databases dictionary with 3 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5

# Modify all electricity generation and market datasets:

This cell uses the Wurst functionality as well as the functions that we wrote in the notebook "_Functions_to_modify_ecoinvent.ipynb" to make the changes to ecoinvent:

In [10]:
for key in pyprind.prog_bar(database_dict.keys()):
    
    
    db = extract_brightway2_databases(['Carma CCS','ecoinvent_3.5'])
    default_global_location(db)
    fix_unset_technosphere_and_production_exchange_locations(db)
    #set_global_location_for_additional_datasets(db)
    remove_nones(db)
    rename_locations(db, fix_names)
    #add_new_locations_to_added_datasets(input_db)
    #regionalize_added_datasets(input_db)
    
    year = database_dict[key]['year']
    scenario = database_dict[key]['scenario']
    remind_data = get_remind_data(scenario)
    print(key)
       
    #Electricity generation datasets:
    print('Changing ecoinvent electricity generation datasets')
    technology_changes = update_electricity_datasets_with_remind_data(db, remind_data, year, agg_func=np.average, update_efficiency = True, update_emissions = False)
    
    # Electricity markets:
    print('Changing electricity Markets')
    market_changes = update_electricity_markets(db, year, remind_data)
    
    #Electricity generation datasets from project Carma
    #print('Changing Carma electricity datasets')
    #modify_all_carma_electricity_datasets(db, year, scenario)
    
    print('Saving changes to excel')
    tech_df = pd.DataFrame.from_dict(technology_changes)
    tech_df.index = pd.MultiIndex.from_tuples(tech_df.index)
    tech_df=tech_df.T
    tech_df = tech_df.set_index([('meta data','remind technology'),('meta data','name'),('meta data','location')], drop=True).sort_index()
    market_df = pd.DataFrame.from_dict(market_changes).T
    market_df = market_df.set_index([('meta data','name'),('meta data','location')], drop=True).sort_index().T

    writer = pd.ExcelWriter('electricity changes '+ str(year) + ' '+ scenario + '.xlsx')
    market_df.to_excel(writer, sheet_name = 'markets')
    for tech in tech_df.index.levels[0]:
        tech_df.loc[tech].dropna(how='all', axis=1).swaplevel(i=0,j=1, axis=1).T.to_excel(writer, sheet_name = tech)
    writer.save()
    del tech_df
    del market_df
    del writer
    
    
    rename_locations(db, fix_names_back)
    if key in databases:
        del databases[key]

    write_brightway2_database(db, key)
    del db
    
        
    lca = LCA({Database(key).random(): 1}, method=lcia_methods['CC'])
    lca.lci()
    lca.lcia()
    print(lca.score)


Getting activity data


  0%|          | 1180/547514 [00:00<00:46, 11794.78it/s]

Adding exchange data to activities


100%|██████████| 547514/547514 [00:31<00:00, 17524.73it/s]


Filling out exchange data


...

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:52


Title: Writing activities to SQLite3 database:
  Started: 11/08/2018 12:11:04
  Finished: 11/08/2018 12:12:56
  Total time elapsed: 00:01:52
  CPU %: 56.80
  Memory %: 22.70
Created database: ecoinvent_Remind_RCP37_2050


0% [######] 100% | ETA: 00:00:00

0.8611866367971789



Total time elapsed: 00:39:47


In [11]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050